# Dendritic Spine Clusterization

## kernel k-Means Zernike Moments

In [ ]:
dataset_folder = '0.025 0.025 0.1 dataset'
lf_metrics_path = f"{dataset_folder}/lf_metrics.csv"
classical_metrics_path = f"{dataset_folder}/classical_metrics.csv"
output_dir = "output/clustering_new_normalized_d=15"
cur_metrics = ["LightFieldZernikeMoments"]
zernike_prefix = f"{output_dir}_zernike_15"
calculate_metrics = False
save_metrics = True
standardize_metrics = True

In [ ]:
%%time
from spine_analysis.mesh.utils import load_spine_meshes
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import create_dir

create_dir(output_dir)
spine_meshes = load_spine_meshes(folder_path=dataset_folder)

if calculate_metrics:
    every_spine_metrics = SpineMetricDataset()
    every_spine_metrics.calculate_metrics(spine_meshes, cur_metrics)

    if save_metrics:
        every_spine_metrics.save(lf_metrics_path)

In [ ]:
from spine_analysis.spine.grouping import SpineGrouping

# load merged and reduced manual classification
manual_classification = SpineGrouping().load(f"{dataset_folder}/manual_classification/manual_classification_merged_reduced.json")

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
import csv
import sys, ctypes as ct

csv.field_size_limit(int(ct.c_ulong(-1).value // 2))
every_spine_metrics = SpineMetricDataset.load(lf_metrics_path)
if standardize_metrics:
    every_spine_metrics.standardize()

## Absolute clusterization

In [ ]:
%%time
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from spine_analysis.clusterization.postprocess import score
from notebook_widgets import kernel_hierarchical_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset

zernike_absolute = every_spine_metrics.get_metrics_subset(['LightFieldZernikeMoments'])
zernike_absolute.clasterization_preprocess(zernike_postprocess='abs')
spine_dataset = SpineMeshDataset().load(dataset_folder)
display(kernel_hierarchical_clustering_experiment_widget(zernike_absolute, every_spine_metrics, spine_dataset, score, metric=LightFieldZernikeMomentsSpineMetric.repr_distance,
                                                         min_num_of_clusters=3,max_num_of_clusters=5, use_pca=False, classification=manual_classification,
                                                         filename_prefix="hi_15_zernike_abs"))

In [ ]:
%%time
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from spine_analysis.clusterization.postprocess import score
from notebook_widgets import kernel_k_means_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset

zernike_absolute = every_spine_metrics.get_metrics_subset(['LightFieldZernikeMoments'])
zernike_absolute.clasterization_preprocess(zernike_postprocess='abs')
spine_dataset = SpineMeshDataset().load(dataset_folder)
display(kernel_k_means_clustering_experiment_widget(zernike_absolute, every_spine_metrics, spine_dataset, score, metric=LightFieldZernikeMomentsSpineMetric.repr_distance,
                                             max_num_of_clusters=9, use_pca=False,
                                             filename_prefix="kk_15_zernike_abs"))

## Real part clusterization

In [ ]:
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from spine_analysis.clusterization.postprocess import score
from notebook_widgets import kernel_k_means_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset

zernike_real = every_spine_metrics.get_metrics_subset(['LightFieldZernikeMoments'])
zernike_real.clasterization_preprocess(zernike_postprocess='real')
# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_folder)
display(kernel_k_means_clustering_experiment_widget(zernike_real, every_spine_metrics, spine_dataset, score, metric=LightFieldZernikeMomentsSpineMetric.repr_distance,
                                             max_num_of_clusters=9, use_pca=False,
                                             filename_prefix="zernike_real_12"))

### LF + classical metrics composition Kernel Hierarchical ABS

In [ ]:
%%time
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from spine_analysis.clusterization.postprocess import score
from notebook_widgets import kernel_hierarchical_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset

zernike_absolute = every_spine_metrics.get_metrics_subset(['LightFieldZernikeMoments'])

classical_spine_metrics = SpineMetricDataset.load(classical_metrics_path)
#classical_spine_metrics = classical_spine_metrics.get_metrics_subset(['Volume'])

spine_idxs = classical_spine_metrics._spine_2_row
for metric_name in classical_spine_metrics.metric_names:
    spines_metric = classical_spine_metrics.column(metric_name)
    zernike_absolute.add_metric({spine: spines_metric[idx] for spine, idx in spine_idxs.items()})

zernike_absolute.clasterization_preprocess(zernike_postprocess='abs')
zernike_absolute.set_metric_distance('LightFieldZernikeMoments', LightFieldZernikeMomentsSpineMetric.repr_distance)

if standardize_metrics:
    zernike_absolute.standardize()

spine_dataset = SpineMeshDataset().load(dataset_folder)

In [ ]:
%%time
from spine_analysis.shape_metric.metrics_composition import SquaredRootComposition
from spine_analysis.clusterization.postprocess import silhouette

import spine_analysis
import importlib
importlib.reload(spine_analysis.clusterization.postprocess)

metrics_composition = SquaredRootComposition(zernike_absolute)

display(kernel_hierarchical_clustering_experiment_widget(zernike_absolute, every_spine_metrics, spine_dataset, silhouette, metric=metrics_composition.distance,
                                                         max_num_of_clusters=9, use_pca=False, classification=manual_classification,
                                                         filename_prefix="hi_square_comp_allclassical_15_zernike_abs"))

In [ ]:
%%javascript
require("notebook/js/notebook").Notebook.prototype.scroll_to_bottom = function () {}